# Aprendizado de Máquina - TP1
# Classificação de Exoplanetas

O trabalho tem como objetivo a prática de conceitos aprendidos na disciplina, adquirindo experiência no uso de métodos de classificação, na avaliação de modelos, e na interpretação e apresentação de resultados de experimentos. Para isso, utilizaremos e compararemos diferentes métodos para solucionar um problema de classificação binária.  

O problema abordado é o da classificação de exoplanetas identificados pela sonda espacial *Kepler* entre **confirmados** e **falsos positivos**. Os possíveis exoplanetas são chamados **Kepler Object of Interest (KOI)**, e cada observação do conjunto de dados corresponde a um KOI e suas características estimadas.

Serão explorados os seguintes métodos de classificação:
- [ ] **Naive Bayes**
    - [ ] Apenas um experimento, para servir de baseline.
- [ ] **Decision Tree**
    - [ ] Variação de altura máxima da árvore, incluindo ilimitada
    - [ ] Visualização gráfica dos resultados
- [ ] **SVM**
    - [ ] Avaliação dos Kernels
        - [ ] Linear
        - [ ] Sigmoid
        - [ ] Polinomial
        - [ ] RBF
- [ ] **k-NN**
    - [ ] Variação do número de vizinhos *k*
    - [ ] Visualização gráfica dos resultados
- [ ] **Random Forest**
    - [ ] Variação do número de árvores
    - [ ] Visualização gráfica dos resultados
- [ ] **Gradient Tree Boosting**
    - [ ] Variação do número de iterações
    - [ ] Visualização gráfica dos resultados

Os métodos estão disponíveis no módulo `scikit-learn`. Iremos utilizar também os módulos `numpy` e `matplotlib.pyplot` para operações matemáticas e geração de gráficos, respectivamente.

In [2]:
import sklearn
import numpy as np
import matplotlib.pyplot as plt